In [ ]:
!pip install pymongo
!pip install requests
!pip install polars
!pip install pandas
!pip install matplotlib
!pip install seaborn
!pip install h2o

In [1]:
import mondongo
import cleanfiles
import eda
import transforms

In [ ]:
mondongo.upload_data()

In [ ]:
cleanfiles.clean_files()

In [2]:
df = eda.get_dataframe()

Conectando a MongoDB para extraer datos...
DataFrame creado con exito: 181691 filas y 136 columnas.


In [ ]:
missing_col = eda.analyze_data_quality(df)

In [ ]:
missing_col

Todas esas columnas tienen un dato vacío o algún nulo

In [ ]:
eda.check_duplicates(df)

No hay ningún valor duplicado

In [ ]:
df.describe()

Analizando las columnas y su contenido vemos que realmente solo tenemos unas 35 columnas útiles

In [3]:
df_clean = eda.select_star_schema_variables(df)

Seleccionando variables para el modelo
Modelo de estrella filtrado: 22 columnas seleccionadas.


In [ ]:
df_clean.schema

Aquí procederemos a modificar el tipo de variable de unas columnas específicas, para que en lugar de estar en string, sean números

In [4]:
df_clean = eda.cast_numeric_columns(df_clean)

Corrigiendo tipos de datos numéricos...
 - Columna 'nkill' convertida a Int64.
 - Columna 'nwound' convertida a Int64.
 - Columna 'propvalue' convertida a Int64.
 - Columna 'iyear' convertida a Int64.
 - Columna 'imonth' convertida a Int64.
 - Columna 'iday' convertida a Int64.
 - Columna 'success' convertida a Int64.
 - Columna 'latitude' convertida a Float64.
 - Columna 'longitude' convertida a Float64.


In [5]:
df_clean.schema

Schema([('nkill', Int64),
        ('nwound', Int64),
        ('success', Int64),
        ('propvalue', Int64),
        ('iyear', Int64),
        ('imonth', Int64),
        ('iday', Int64),
        ('country_txt', String),
        ('region_txt', String),
        ('provstate', String),
        ('city', String),
        ('latitude', Float64),
        ('longitude', Float64),
        ('gname', String),
        ('gsubname', String),
        ('attacktype1_txt', String),
        ('suicide', String),
        ('targtype1_txt', String),
        ('corp1', String),
        ('target1', String),
        ('weaptype1_txt', String),
        ('weapsubtype1_txt', String)])

Y ahora procederemos con el mapeo de las columnas categóricas.

In [6]:
eda.list_categorical_uniques(df_clean)

Buscando valores únicos en columnas de texto...

Columna String: 'country_txt'
  Total únicos: 205
  Valores (Top 15): ['Zaire', 'Vanuatu', 'China', 'Honduras', 'Czech Republic', 'Zimbabwe', 'Tajikistan', 'Ukraine', 'Sierra Leone', 'Georgia', 'Italy', 'Yugoslavia', 'Portugal', 'Cyprus', 'French Guiana']...

Columna String: 'region_txt'
  Total únicos: 12
  Valores: ['Southeast Asia', 'Eastern Europe', 'Central Asia', 'Middle East & North Africa', 'Sub-Saharan Africa', 'Central America & Caribbean', 'South Asia', 'North America', 'South America', 'East Asia', 'Western Europe', 'Australasia & Oceania']

Columna String: 'provstate'
  Total únicos: 2855
  Valores (Top 15): ['Bishkek (Province)', 'Granada', 'British Columbia', 'Kuala Lumpur', 'Lautem (District)', 'Croatia', 'Boumerdes (Province)', 'Santa Cruz', 'Golan Heights', 'Sakhalin (Oblast)', 'Kagera', 'Stara Zagora', 'Kakamega', 'unknown', 'Perlis']...

Columna String: 'city'
  Total únicos: 36674
  Valores (Top 15): ['Cabunbata', 'C

{'country_txt': ['Zaire',
  'Vanuatu',
  'China',
  'Honduras',
  'Czech Republic',
  'Zimbabwe',
  'Tajikistan',
  'Ukraine',
  'Sierra Leone',
  'Georgia',
  'Italy',
  'Yugoslavia',
  'Portugal',
  'Cyprus',
  'French Guiana',
  'Mozambique',
  'Fiji',
  'Serbia-Montenegro',
  'Estonia',
  'Belarus',
  'North Korea',
  'Chile',
  'Algeria',
  'Niger',
  'Nepal',
  'Sri Lanka',
  'Seychelles',
  'Guinea-Bissau',
  'Gabon',
  'Taiwan',
  'Iraq',
  'Slovak Republic',
  'Morocco',
  'Guatemala',
  'Latvia',
  'Costa Rica',
  'Uzbekistan',
  'Norway',
  'Wallis and Futuna',
  'Chad',
  'Equatorial Guinea',
  'New Caledonia',
  'Czechoslovakia',
  'Macedonia',
  'South Vietnam',
  'New Hebrides',
  'Luxembourg',
  'Central African Republic',
  'Ethiopia',
  'Rwanda',
  'Switzerland',
  'West Germany (FRG)',
  'France',
  'Libya',
  'Montenegro',
  'Kuwait',
  'Burkina Faso',
  'Togo',
  'Romania',
  'Lesotho',
  'St. Kitts and Nevis',
  'Moldova',
  'Lebanon',
  'Nigeria',
  'Comoros',
  

Con esas columnas que sabemos que son tipo string, vamos a codificarlas a numérico conservando el mapeo.

In [7]:
df_num, mis_mapeos = eda.encode_categorical_columns(df_clean)

Codificando columnas detectadas como texto: ['country_txt', 'region_txt', 'provstate', 'city', 'gname', 'gsubname', 'attacktype1_txt', 'suicide', 'targtype1_txt', 'corp1', 'target1', 'weaptype1_txt', 'weapsubtype1_txt']...
 - 'country_txt' codificado (205 categorías).
 - 'region_txt' codificado (12 categorías).
 - 'provstate' codificado (2856 categorías).
 - 'city' codificado (36675 categorías).
 - 'gname' codificado (3537 categorías).
 - 'gsubname' codificado (1184 categorías).
 - 'attacktype1_txt' codificado (9 categorías).
 - 'suicide' codificado (2 categorías).
 - 'targtype1_txt' codificado (22 categorías).
 - 'corp1' codificado (33242 categorías).
 - 'target1' codificado (86009 categorías).
 - 'weaptype1_txt' codificado (12 categorías).
 - 'weapsubtype1_txt' codificado (31 categorías).


Para poder trabajar con h2o, este solo acepta dataframes de pandas, por lo tanto creamos una funcion para convertir el dataframe de polars a pandas.

In [8]:
df_pandas = transforms.convert_to_pandas(df_num)

Iniciando conversión de Polars a Pandas...
Conversión exitosa. DataFrame de Pandas listo con 181691 registros.


In [10]:
df_pandas

,nkill,nwound,success,propvalue,iyear,imonth,iday,country_txt,region_txt,provstate,...,longitude,gname,gsubname,attacktype1_txt,suicide,targtype1_txt,corp1,target1,weaptype1_txt,weapsubtype1_txt
0,1,0,1,0,1970,7,2,195,0,NaN,...,-69.951164,3135,NaN,1,0,14,NaN,17515.0,8,NaN
1,0,0,1,0,1970,0,0,36,11,337.0,...,-99.086624,2144,NaN,4,0,3,6185.0,66529.0,8,NaN
2,1,0,1,0,1970,1,0,29,2,1558.0,...,120.599741,1833,NaN,1,0,20,2429.0,39899.0,8,NaN
3,0,0,1,0,1970,1,0,134,10,619.0,...,23.762728,1833,NaN,7,0,3,NaN,39441.0,10,16.0
4,0,0,1,0,1970,1,0,187,1,2072.0,...,130.396361,1833,NaN,8,0,3,NaN,49897.0,7,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181686,1,2,1,0,2017,12,31,20,8,681.0,...,45.385034,1749,NaN,5,0,10,12568.0,66852.0,5,0.0
181687,2,7,1,-99,2017,12,31,143,7,317.0,...,35.942679,1791,NaN,7,0,10,24980.0,61332.0,10,9.0
181688,0,0,1,-99,2017,12,31,29,2,2041.0,...,124.437908,2948,NaN,8,0,14,2400.0,14159.0,7,7.0
181689,0,0,0,0,2017,12,31,105,6,1534.0,...,93.940430,1833,NaN,7,0,0,21467.0,44726.0,10,19.0
